In [4]:
import pandas as pd

# Загрузка данных из файла Excel
file_path = '/Users/Gorin/Desktop/Тест/tasks_2025-02-04_15_23_27.xls'  # Укажи путь к своему файлу
df = pd.read_excel(file_path)

# Проверка на наличие нужных столбцов
if 'Постановщик' not in df.columns or 'Исполнитель' not in df.columns:
    raise ValueError("Отсутствуют необходимые столбцы 'Постановщик' или 'Исполнитель'.")

# Заполнение пустых ячеек значением 'пусто'
df['Постановщик'].fillna('пусто', inplace=True)
df['Исполнитель'].fillna('пусто', inplace=True)

# Подсчитываем количество обращений для каждого постановщика
result_postanovshik = df['Постановщик'].value_counts().reset_index()
result_postanovshik.columns = ['Постановщик', 'Сумма задач']
result_postanovshik.insert(0, 'Индекс', range(1, len(result_postanovshik) + 1))

# Подсчитываем количество задач для каждого исполнителя
result_executor = df['Исполнитель'].value_counts().reset_index()
result_executor.columns = ['Исполнитель', 'Сумма задач']
result_executor.insert(0, 'Индекс', range(1, len(result_executor) + 1))

# Суммирование в первых двух сводных таблицах
sum_postanovshik = pd.DataFrame({'Постановщик': ['Сумма'], 'Сумма задач': [result_postanovshik['Сумма задач'].sum()]})
sum_executor = pd.DataFrame({'Исполнитель': ['Сумма'], 'Сумма задач': [result_executor['Сумма задач'].sum()]})

# Добавляем строку с суммой в обе таблицы
result_postanovshik = pd.concat([result_postanovshik, sum_postanovshik], ignore_index=True)
result_executor = pd.concat([result_executor, sum_executor], ignore_index=True)

# Преобразуем индексы в целые числа, заменяя NaN на 0
result_postanovshik['Индекс'] = result_postanovshik['Индекс'].fillna(0).astype(int)
result_executor['Индекс'] = result_executor['Индекс'].fillna(0).astype(int)

# Создание столбца для времени выполнения задачи
df['Время выполнения задачи'] = (df['Дата закрытия'] - df['Дата создания']).dt.total_seconds() / 3600  # Время в часах

# Округление времени выполнения задач до тысячных
df['Время выполнения задачи'] = df['Время выполнения задачи'].round(3)

# Группируем по исполнителю и вычисляем среднее время выполнения задачи
average_time_per_executor = df.groupby('Исполнитель')['Время выполнения задачи'].mean().reset_index()

# Создаем Excel файл с четырьмя листами
with pd.ExcelWriter('/Users/Gorin/Desktop/результаты_сводных_таблиц.xlsx') as writer:
    # Записываем сводную таблицу по постановщикам
    result_postanovshik.to_excel(writer, sheet_name='Постановщики', index=False)
    
    # Записываем сводную таблицу по исполнителям
    result_executor.to_excel(writer, sheet_name='Исполнители', index=False)
    
    # Записываем список с исполнителями и временем выполнения задач на новый лист "Время выполнения"
    df[['Исполнитель', 'Время выполнения задачи']].to_excel(writer, sheet_name='Время выполнения', index=False)
    
    # Записываем среднее время выполнения по каждому исполнителю на новый лист "Среднее время выполнения"
    average_time_per_executor.to_excel(writer, sheet_name='Среднее время выполнения', index=False)

print("Результаты успешно сохранены в файл 'результаты_сводных_таблиц.xlsx'")


Результаты успешно сохранены в файл 'результаты_сводных_таблиц.xlsx'
